<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/agent_design_patterns_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install crewai and langchain-openai
!pip install crewai langchain-openai -q
!pip install pydantic  -q # Ensure pydantic is installed
!pip install 'crewai[tools]' -q

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool # Import BaseTool
from pydantic import BaseModel, Field # Import BaseModel and Field for schema definition
from litellm import litellm # Import litellm directly
from google.colab import userdata # For Google Colab environment

In [8]:
!pip show crewai-tools

Name: crewai-tools
Version: 0.47.1
Summary: Set of tools for the crewAI framework
Home-page: https://crewai.com
Author: 
Author-email: João Moura <joaomdmoura@gmail.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: chromadb, click, crewai, docker, embedchain, lancedb, openai, pydantic, pyright, pytube, requests, tiktoken
Required-by: 


In [2]:
import os
import warnings # Import the warnings module

# Suppress warnings that are not critical to code logic
warnings.filterwarnings("ignore", category=DeprecationWarning, module="httpx")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipywidgets")

from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from litellm import litellm
from google.colab import userdata

# Import necessary Langchain components for custom LLM integration
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.outputs import ChatResult, Generation
from typing import Any, List, Dict, Optional, Type

# --- 1. Custom LiteLLM Wrapper for CrewAI ---
class LiteLLMChatModel(BaseChatModel):
    """
    A custom ChatModel for Langchain (and thus CrewAI) that uses LiteLLM.
    """
    model_name: str
    api_key: str
    temperature: float = 0.7
    max_tokens: Optional[int] = None

    def __init__(self, model_name: str, api_key: str, temperature: float = 0.7, max_tokens: Optional[int] = None, **kwargs: Any):
        super().__init__(model_name=model_name, api_key=api_key, temperature=temperature, max_tokens=max_tokens, **kwargs)
        self.model_name = model_name
        self.api_key = api_key
        self.temperature = temperature
        self.max_tokens = max_tokens

    def _generate(
        self,
        messages: List[Any],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> ChatResult:
        litellm_messages = []
        for message in messages:
            if isinstance(message, HumanMessage):
                litellm_messages.append({"role": "user", "content": message.content})
            elif isinstance(message, AIMessage):
                litellm_messages.append({"role": "assistant", "content": message.content})
            elif isinstance(message, SystemMessage):
                litellm_messages.append({"role": "system", "content": message.content})
            else:
                raise ValueError(f"Unsupported message type: {type(message)}")

        try:
            response = litellm.completion(
                model=self.model_name,
                messages=litellm_messages,
                api_key=self.api_key,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                stop=stop,
                **kwargs
            )

            response_content = response.choices[0].message.content
            ai_message = AIMessage(content=response_content)
            generation = Generation(text=response_content, message=ai_message)
            return ChatResult(generations=[generation])

        except Exception as e:
            print(f"LiteLLM call failed: {e}")
            raise e

    @property
    def _llm_type(self) -> str:
        return "litellm_chat_model"

    def _get_parameters(self) -> Dict[str, Any]:
        return {
            "model_name": self.model_name,
            "api_key": self.api_key,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
        }

    def _stream(self, messages: List[Any], stop: Optional[List[str]] = None, **kwargs: Any) -> Any:
        raise NotImplementedError("LiteLLMChatModel does not support streaming yet.")


# --- Configuration for DeepSeek LLM ---
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
my_deepseek_llm = LiteLLMChatModel(
    model_name="deepseek/deepseek-reasoner",
    api_key=deepseek_api_key,
    temperature=0.5,
    max_tokens=4096, # Ensure this is an adequate value for DeepSeek responses
    request_timeout=120 #
)

# --- Define Pydantic Models for Tool Inputs ---
class FlightInput(BaseModel):
    """Input for FlightAPIQueryTool."""
    origin: str = Field(description="The departure city or airport code.")
    destination: str = Field(description="The arrival city or airport code.")
    trip_dates: str = Field(description="The dates of the trip, e.g., 'next month', 'July 15-22'.")
    passengers: int = Field(description="Number of passengers.", default=1)

class HotelInput(BaseModel):
    """Input for HotelAPIQueryTool."""
    location: str = Field(description="The city or area for the hotel search.")
    check_in_date: str = Field(description="The check-in date, e.g., 'July 15'.")
    check_out_date: str = Field(description="The check-out date, e.g., 'July 22'.")
    preferences: Optional[str] = Field(description="Any specific preferences like 'luxury', 'budget', 'near airport'.", default="")


# --- Refactor Custom Tools using BaseTool and Pydantic ---

class FlightAPIQueryTool(BaseTool):
    name: str = "Flight API Query Tool"
    description: str = (
        "Queries a simulated flight booking API for flight details. "
        "Use this tool when the user asks for flight information including origin, destination, and dates."
    )
    args_schema: Type[BaseModel] = FlightInput

    def _run(self, origin: str, destination: str, trip_dates: str, passengers: int = 1) -> str:
        print(f"\n--- Using {self.name} with origin='{origin}', destination='{destination}', dates='{trip_dates}', passengers={passengers} ---")
        if "London" in origin and "New York" in destination:
            return f"Found direct flights: BA123 ({origin}-{destination}), AA456 ({origin}-{destination}). Prices start at $550 for {trip_dates}."
        elif "Montreal" in origin and "Paris" in destination:
            return f"Found flights with layovers: AC870 ({origin}-{destination} via Toronto). Prices around $700 for {trip_dates}."
        else:
            return "No direct flights found for the specified route. Suggesting alternatives."

class HotelAPIQueryTool(BaseTool):
    name: str = "Hotel API Query Tool"
    description: str = (
        "Queries a simulated hotel booking API for hotel options. "
        "Use this tool when the user asks for hotel information including location, check-in/out dates, and preferences."
    )
    args_schema: Type[BaseModel] = HotelInput

    def _run(self, location: str, check_in_date: str, check_out_date: str, preferences: Optional[str] = "") -> str:
        print(f"\n--- Using {self.name} with location='{location}', check-in='{check_in_date}', check-out='{check_out_date}', preferences='{preferences}' ---")
        if "New York" in location and "luxury" in preferences:
            return f"Luxury hotels in NYC for {check_in_date}-{check_out_date}: The Plaza, Mandarin Oriental. Average $600/night."
        elif "Paris" in location and "budget" in preferences:
            return f"Budget hotels in Paris for {check_in_date}-{check_out_date}: Ibis Styles, Generator Hostel. Average $100/night."
        else:
            return "No hotels matching criteria."

# Initialize the primary tools
flight_api_tool = FlightAPIQueryTool()
hotel_api_tool = HotelAPIQueryTool()

# --- Optional Real Web Search Tools  ---
# Ensure you have SERPER_API_KEY set in your Colab userdata
# and run `!pip install 'crewai-tools[website-reader]'` for WebsiteReadTool
from crewai_tools import SerperDevTool
# Robust import for WebsiteReadTool with a fallback
WebsiteReadTool_imported = None
try:
    from crewai_tools.tools import WebsiteReadTool as WebsiteReadTool_candidate
    WebsiteReadTool_imported = WebsiteReadTool_candidate
except ImportError:
    pass # Try next path
if not WebsiteReadTool_imported:
    try:
        from crewai_tools.tool_libs.website_loader import WebsiteReadTool as WebsiteReadTool_candidate
        WebsiteReadTool_imported = WebsiteReadTool_candidate
    except ImportError:
        pass # Not found, remain None

general_search_tools = []
try:
    serper_tool = SerperDevTool()
    general_search_tools.append(serper_tool)
    if WebsiteReadTool_imported: # Only add if successfully imported
        website_read_tool = WebsiteReadTool_imported()
        general_search_tools.append(website_read_tool)
    print("General web search tools (SerperDevTool, WebsiteReadTool) initialized successfully.")
except Exception as e:
    print(f"Warning: General web search tools could not be initialized: {e}. "
          "Make sure SERPER_API_KEY is set and 'crewai-tools[website-reader]' is installed.")
    # general_search_tools remains empty if initialization fails


# --- Define Agents ---
class FlightPlanningAgents:
    def __init__(self, llm_model: BaseChatModel):
        self.llm = llm_model

    def flight_researcher(self):
        return Agent(
            role='Flight Information Researcher',
            goal='Find the best flight options based on user query, including cheapest, fastest, and most convenient routes.',
            backstory="""You are an expert travel agent specializing in flight research. You have access to vast flight databases and can quickly identify optimal routes, prices and airlines. You are meticulous and always aim to find the best value for the customer.""",
            verbose=False ,
            allow_delegation=False,
            tools=[flight_api_tool] + general_search_tools, # Assign real search tools if available
            llm=self.llm
        )

    def hotel_researcher(self):
        return Agent(
            role='Hotel and Accommodation Researcher',
            goal='Identify suitable hotel options that align with user preferences (e.g., budget, luxury, location).',
            backstory="""You are a seasoned hospitality expert with an encyclopedic knowledge of hotels worldwide. You can quickly filter options based on budget, amenities, and location to provide the perfect stay.""",
            verbose=False ,
            allow_delegation=False,
            tools=[hotel_api_tool] + general_search_tools, # Assign real search tools if available
            llm=self.llm
        )

    def itinerary_builder(self):
        return Agent(
            role='Personalized Itinerary Creator',
            goal='Synthesize flight and hotel information into a coherent, personalized travel itinerary for the user.',
            backstory="""You are a master of travel logistics, capable of weaving together disparate travel details into a seamless, easy-to-follow itinerary. Your goal is to make travel planning effortless for the user.""",
            verbose=False ,
            allow_delegation=True,
            llm=self.llm
        )

    def travel_router(self):
        return Agent(
            role='Travel Query Router',
            goal='Determine the primary intent of a user\'s travel query and route it to the appropriate specialized agent (flight, hotel, or itinerary).',
            backstory="""You are the first point of contact for all travel inquiries. Your sharp analytical skills allow you to quickly understand a user's core need and efficiently direct their request to the most suitable travel expert in your team.""",
            verbose=False ,
            allow_delegation=False,
            llm=self.llm
        )


# --- Define Tasks ---
class FlightPlanningTasks:
    def __init__(self):
        pass

    def research_flight_options(self, agent: Agent, query: str):
        return Task(
            description=f"Thoroughly research flight options for the following request: '{query}'. "
                        "Identify the origin, destination, and dates from the query to accurately use the flight tool. "
                        "Focus on finding direct flights, reasonable layovers if direct isn't possible, "
                        "and provide a range of prices. If direct flights are not found, use web search tools to find alternative routes or general travel advice. ",
            expected_output="A concise summary of top 2-3 flight options including airlines, dates, times, and estimated prices for the query.",
            agent=agent,
            tools=[flight_api_tool] + general_search_tools # Tools at task level too for clarity
        )

    def research_hotel_options(self, agent: Agent, query: str):
        return Task(
            description=f"Investigate hotel options relevant to: '{query}'. "
                        "Extract the location, check-in/out dates, and any preferences from the query to use the hotel tool. "
                        "Identify top 2-3 hotel suggestions with approximate costs and key features. Use web search tools for general info about the location if needed.",
            expected_output="A list of 2-3 suitable hotels with brief descriptions, estimated nightly rates, and relevant amenities for the query.",
            agent=agent,
            tools=[hotel_api_tool] + general_search_tools # Tools at task level too for clarity
        )

    def create_travel_itinerary(self, agent: Agent, original_query: str, context_tasks: List[Task]):
        return Task(
            description=f"Compile a comprehensive travel itinerary based on the original query: '{original_query}'. "
                        "Access the flight information from the context of previous tasks. "
                        "Access the hotel information from the context of previous tasks. "
                        "Synthesize this information. Structure the itinerary clearly, perhaps day-by-day, suggesting activities or key travel points. If information is missing, ask for clarification or use available tools to find general travel advice.",
            expected_output="A well-structured, detailed travel itinerary in markdown format, combining flight, hotel, and general travel advice. It should be easy for a user to understand and follow.",
            agent=agent,
            context=context_tasks
        )

    def route_travel_query(self, agent: Agent, query: str):
        return Task(
            description=f"Analyze the user's travel query: '{query}'. "
                        "Determine if it's primarily a flight search, a hotel search, or a request for a full itinerary. "
                        "Output the identified intent (e.g., 'flight_search', 'hotel_search', 'full_itinerary') "
                        "and any key details extracted from the query, specifically, "
                        "for flight search: 'origin', 'destination', 'trip_dates', 'passengers'. "
                        "For hotel search: 'location', 'check_in_date', 'check_out_date', 'preferences'. "
                        "For full itinerary: 'destination', 'duration', and whether flights/hotels are needed.",
            expected_output="A JSON string indicating the intent and extracted parameters, e.g., "
                            "{'intent': 'flight_search', 'args': {'origin': 'New York', 'destination': 'London', 'trip_dates': 'next month'}}",
            agent=agent
        )


# --- Orchestrate with CrewAI ---

class FlightPlanningCrew:
    def __init__(self, llm_model: BaseChatModel):
        self.agents = FlightPlanningAgents(llm_model)
        self.tasks = FlightPlanningTasks()
        self.llm_model = llm_model

    def create_sequential_flight_planner_crew(self):
        flight_researcher = self.agents.flight_researcher()
        hotel_researcher = self.agents.hotel_researcher()
        itinerary_builder = self.agents.itinerary_builder()

        task_flight_research = self.tasks.research_flight_options(
            agent=flight_researcher,
            query="I need a flight from Montreal (YUL) to Paris (CDG) for July 15-22, 2025 for 2 people."
        )

        task_hotel_research = self.tasks.research_hotel_options(
            agent=hotel_researcher,
            query="I need a mid-range hotel in Paris for July 15-22, 2025."
        )

        task_build_itinerary = self.tasks.create_travel_itinerary(
            agent=itinerary_builder,
            original_query="Montreal to Paris 7-day trip",
            context_tasks=[task_flight_research, task_hotel_research]
        )

        crew = Crew(
            agents=[flight_researcher, hotel_researcher, itinerary_builder],
            tasks=[task_flight_research, task_hotel_research, task_build_itinerary],
            process=Process.sequential,
            verbose=False
        )
        return crew

    def create_router_based_flight_planner_crew(self, user_query: str):
        travel_router = self.agents.travel_router()
        flight_researcher = self.agents.flight_researcher()
        hotel_researcher = self.agents.hotel_researcher()
        itinerary_builder = self.agents.itinerary_builder()

        task_route_query = self.tasks.route_travel_query(
            agent=travel_router,
            query=user_query
        )

        crew = Crew(
            agents=[travel_router, flight_researcher, hotel_researcher, itinerary_builder],
            tasks=[task_route_query],
            process=Process.hierarchical,
            manager_llm=self.llm_model,
            verbose=False
        )
        return crew


# --- Run the Crews ---
if __name__ == "__main__":
    print("Welcome to your DeepSeek-powered AI agent for flight planning!\n")

    crew_builder = FlightPlanningCrew(my_deepseek_llm)

    # --- Demo 1: Sequential Flight Planning ---
    print("\n" + "="*20 + " Demo 1: Sequential Flight Planning " + "="*20)
    sequential_crew = crew_builder.create_sequential_flight_planner_crew()
    print("\nStarting Sequential Flight Planning Crew...")
    try:
        result_sequential = sequential_crew.kickoff()
        print("\n## Sequential Flight Planning Result:")
        print(result_sequential)
    except Exception as e:
        print(f"\nError running sequential crew: {e}")
    print("\n" + "="*60 + "\n")

    # --- Demo 2: Router-based Flight Planning ---
    print("\n" + "="*20 + " Demo 2: Router-based Flight Planning " + "="*20)
    flight_query = "Find me a cheap flight from London Heathrow (LHR) to New York (JFK) for next month for 1 person."
    router_crew_flight = crew_builder.create_router_based_flight_planner_crew(flight_query)
    print(f"\nStarting Router-based Flight Planning Crew for: '{flight_query}'")
    try:
        result_router_flight = router_crew_flight.kickoff()
        print("\n## Router-based Flight Planning Result (Flight Query):")
        print(result_router_flight)
    except Exception as e:
        print(f"\nError running router crew (flight query): {e}")
    print("\n" + "="*60 + "\n")

General web search tools (SerperDevTool, WebsiteReadTool) initialized successfully.
Welcome to your DeepSeek-powered AI agent for flight planning!


==================== Demo 1: Sequential Flight Planning ====================

Starting Sequential Flight Planning Crew...

--- Using Flight API Query Tool with origin='YUL', destination='CDG', dates='July 15-22, 2025', passengers=2 ---

--- Using Hotel API Query Tool with location='Paris', check-in='July 15, 2025', check-out='July 22, 2025', preferences='mid-range' ---

--- Using Hotel API Query Tool with location='Paris', check-in='July 15, 2025', check-out='July 22, 2025', preferences='' ---

## Sequential Flight Planning Result:
```markdown
# PARIS TRAVEL ITINERARY: 7 DAYS (JULY 15-22, 2025)

## ✈️ FLIGHT DETAILS  
**Selected Option:** Air Canada Direct (Recommended)  
- **Departure:** July 15, 2025 | Montréal (YUL) 20:55 → Paris (CDG) 09:45+1  
- **Return:** July 22, 2025 | Paris (CDG) 11:30 → Montréal (YUL) 13:35  
- **Duration:** 7h 